In [ ]:
!pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple
!pip install paddleocr
!pip install matplotlib opencv-python tqdm
!pip install paddlepaddle


In [ ]:
!git clone https://github.com/huynhdang0987/DLFinal.git


In [ ]:

import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from tqdm import tqdm
from collections import Counter
import pandas as pd
from datetime import datetime
import io
from PIL import Image
import matplotlib.patches as patches
from google.colab import files
from paddleocr import PaddleOCR, draw_ocr

In [ ]:
BASE_DIR = '/content/DLFinal'
IMAGE_DIR = os.path.join(BASE_DIR, 'images')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')
VISUALIZE_DIR = os.path.join(OUTPUT_DIR, 'visualized')

# Tạo thư mục đầu ra nếu chưa tồn tại
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(VISUALIZE_DIR, exist_ok=True)

In [ ]:
def upload_images():
    print("Hãy tải lên các hình ảnh cần gán nhãn:")
    uploaded = files.upload()

    if not os.path.exists(IMAGE_DIR):
        os.makedirs(IMAGE_DIR)

    for filename, content in uploaded.items():
        with open(os.path.join(IMAGE_DIR, filename), 'wb') as f:
            f.write(content)

    print(f"Đã tải lên {len(uploaded)} hình ảnh vào {IMAGE_DIR}")


In [ ]:
def auto_label_with_paddleocr(image_dir, output_json, lang='vi', visualize_dir=None):
    """
    Sử dụng PaddleOCR để tự động gán nhãn cho các hình ảnh

    Args:
        image_dir: Thư mục chứa hình ảnh
        output_json: Đường dẫn để lưu file JSON kết quả
        lang: Ngôn ngữ cho OCR ('vi' cho tiếng Việt, 'en' cho tiếng Anh)
        visualize_dir: Thư mục để lưu hình ảnh với visualization
    """
    print(f"Bắt đầu gán nhãn tự động với ngôn ngữ: {lang}")

    # Khởi tạo PaddleOCR với ngôn ngữ đã chọn
    ocr = PaddleOCR(use_angle_cls=True, lang=lang)

    dataset = {
        "images": []
    }

    image_id = 1
    annotation_id = 1

    # Lấy danh sách file ảnh
    image_files = [f for f in os.listdir(image_dir)
                  if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.bmp'))]

    if len(image_files) == 0:
        print(f"Không tìm thấy ảnh trong thư mục {image_dir}")
        return

    print(f"Tìm thấy {len(image_files)} hình ảnh để xử lý")

    # Lặp qua tất cả các file trong thư mục hình ảnh
    for filename in tqdm(image_files, desc="Gán nhãn ảnh"):
        image_path = os.path.join(image_dir, filename)

        # Đọc hình ảnh
        try:
            img = cv2.imread(image_path)
            if img is None:
                print(f"Không thể đọc hình ảnh: {image_path}, bỏ qua.")
                continue

            height, width, _ = img.shape
        except Exception as e:
            print(f"Lỗi khi đọc hình ảnh {filename}: {str(e)}")
            continue

        # Tạo entry cho hình ảnh
        image_entry = {
            "file_name": filename,
            "width": width,
            "height": height,
            "id": image_id,
            "annotations": []
        }

        # Thực hiện OCR
        try:
            result = ocr.ocr(image_path, cls=True)

            # Xử lý kết quả OCR
            if result and result[0]:
                for line in result[0]:
                    if line:
                        box, (text, confidence) = line

                        # Chuyển đổi box từ [[x1,y1],[x2,y2],[x3,y3],[x4,y4]] sang [x,y,width,height]
                        box = np.array(box).astype(np.int32)
                        x, y = box.min(axis=0)
                        max_x, max_y = box.max(axis=0)
                        width_box = max_x - x
                        height_box = max_y - y

                        # Tạo annotation
                        annotation = {
                            "id": annotation_id,
                            "bbox": [int(x), int(y), int(width_box), int(height_box)],
                            "text": text,
                            "confidence": float(confidence),
                            "language": lang,
                            "polygon": [[int(p[0]), int(p[1])] for p in box]  # Lưu lại đa giác gốc
                        }

                        image_entry["annotations"].append(annotation)
                        annotation_id += 1
        except Exception as e:
            print(f"Lỗi khi xử lý OCR cho {filename}: {str(e)}")

        dataset["images"].append(image_entry)
        image_id += 1

        # Visualization (tùy chọn)
        if visualize_dir and result and result[0]:
            try:
                # Sử dụng hàm visualization có sẵn của PaddleOCR
                image = Image.open(image_path).convert('RGB')
                boxes = [line[0] for line in result[0]]
                txts = [line[1][0] for line in result[0]]
                scores = [line[1][1] for line in result[0]]

                # Vẽ kết quả OCR
                im_show = draw_ocr(image, boxes, txts, scores, font_path='./fonts/simfang.ttf')
                im_show = Image.fromarray(im_show)

                # Lưu hình ảnh
                output_img_path = os.path.join(visualize_dir, f"vis_{filename}")
                im_show.save(output_img_path)
            except Exception as e:
                print(f"Lỗi khi tạo visualization cho {filename}: {str(e)}")

    # Lưu dataset vào file JSON
    try:
        with open(output_json, 'w', encoding='utf-8') as f:
            json.dump(dataset, f, ensure_ascii=False, indent=2)
        print(f"Đã gán nhãn tự động cho {image_id-1} hình ảnh và lưu vào {output_json}")
    except Exception as e:
        print(f"Lỗi khi lưu JSON: {str(e)}")

    return dataset



In [ ]:
def evaluate_dataset(dataset=None, json_path=None):
    """
    Đánh giá thống kê về bộ dataset

    Args:
        dataset: Dữ liệu JSON đã tải (ưu tiên)
        json_path: Đường dẫn đến file JSON chứa annotations (sử dụng nếu dataset=None)
    """
    # Đọc dataset nếu chưa cung cấp
    if dataset is None and json_path:
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                dataset = json.load(f)
        except Exception as e:
            print(f"Lỗi khi đọc file JSON: {str(e)}")
            return

    if not dataset:
        print("Không có dữ liệu để đánh giá!")
        return

    # Thống kê cơ bản
    num_images = len(dataset["images"])
    total_annotations = 0
    text_lengths = []
    bbox_sizes = []
    bbox_ratios = []  # Tỉ lệ width/height
    confidences = []
    languages = Counter()
    chars_frequency = Counter()

    # Thống kê số lượng annotations theo hình ảnh
    annotations_per_image = []

    for image_data in dataset["images"]:
        annotations = image_data["annotations"]
        num_annotations = len(annotations)
        annotations_per_image.append(num_annotations)
        total_annotations += num_annotations

        for ann in annotations:
            # Thống kê độ dài văn bản
            text = ann["text"]
            text_lengths.append(len(text))

            # Thống kê tần suất các ký tự
            for char in text:
                chars_frequency[char] += 1

            # Thống kê kích thước bounding box
            bbox = ann["bbox"]
            area = bbox[2] * bbox[3]  # width * height
            bbox_sizes.append(area)

            # Tỉ lệ width/height
            if bbox[3] > 0:  # Tránh chia cho 0
                ratio = bbox[2] / bbox[3]
                bbox_ratios.append(ratio)

            # Thống kê confidence
            if "confidence" in ann:
                confidences.append(ann["confidence"])

            # Thống kê ngôn ngữ
            if "language" in ann:
                languages[ann["language"]] += 1

    # In kết quả thống kê
    print("\n=== THỐNG KÊ BỘ DATASET ===")
    print(f"Tổng số hình ảnh: {num_images}")
    print(f"Tổng số annotations: {total_annotations}")
    print(f"Trung bình annotations mỗi hình: {total_annotations / max(num_images, 1):.2f}")

    if text_lengths:
        print(f"Độ dài văn bản:")
        print(f"  - Trung bình: {sum(text_lengths) / len(text_lengths):.2f} ký tự")
        print(f"  - Nhỏ nhất: {min(text_lengths)} ký tự")
        print(f"  - Lớn nhất: {max(text_lengths)} ký tự")

    if bbox_sizes:
        print(f"Kích thước bounding box:")
        print(f"  - Trung bình: {sum(bbox_sizes) / len(bbox_sizes):.2f} pixels")
        print(f"  - Nhỏ nhất: {min(bbox_sizes):.2f} pixels")
        print(f"  - Lớn nhất: {max(bbox_sizes):.2f} pixels")

    if bbox_ratios:
        print(f"Tỉ lệ width/height:")
        print(f"  - Trung bình: {sum(bbox_ratios) / len(bbox_ratios):.2f}")
        print(f"  - Nhỏ nhất: {min(bbox_ratios):.2f}")
        print(f"  - Lớn nhất: {max(bbox_ratios):.2f}")

    if confidences:
        print(f"Confidence:")
        print(f"  - Trung bình: {sum(confidences) / len(confidences):.4f}")
        print(f"  - Nhỏ nhất: {min(confidences):.4f}")
        print(f"  - Lớn nhất: {max(confidences):.4f}")

    if languages:
        print(f"Phân bố ngôn ngữ: {dict(languages)}")

    # Top ký tự xuất hiện nhiều nhất
    print("\nTop 10 ký tự xuất hiện nhiều nhất:")
    for char, count in chars_frequency.most_common(10):
        if char.strip():  # Bỏ qua các ký tự trống
            print(f"  '{char}': {count} lần")

    # Vẽ các biểu đồ thống kê
    plt.figure(figsize=(16, 12))

    # 1. Histogram về độ dài văn bản
    plt.subplot(2, 2, 1)
    plt.hist(text_lengths, bins=20, color='skyblue', edgecolor='black')
    plt.title('Phân bố độ dài văn bản')
    plt.xlabel('Số ký tự')
    plt.ylabel('Số lượng')
    plt.grid(alpha=0.3)

    # 2. Histogram về kích thước bounding box
    plt.subplot(2, 2, 2)
    plt.hist(bbox_sizes, bins=20, color='lightgreen', edgecolor='black')
    plt.title('Phân bố kích thước bounding box')
    plt.xlabel('Diện tích (pixels)')
    plt.ylabel('Số lượng')
    plt.grid(alpha=0.3)

    # 3. Histogram về tỉ lệ width/height
    if bbox_ratios:
        plt.subplot(2, 2, 3)
        plt.hist(bbox_ratios, bins=20, color='salmon', edgecolor='black')
        plt.title('Phân bố tỉ lệ width/height của bounding box')
        plt.xlabel('Tỉ lệ width/height')
        plt.ylabel('Số lượng')
        plt.grid(alpha=0.3)

    # 4. Histogram về số lượng annotations mỗi ảnh
    plt.subplot(2, 2, 4)
    plt.hist(annotations_per_image, bins=range(max(annotations_per_image)+2),
             color='mediumpurple', edgecolor='black')
    plt.title('Phân bố số lượng annotations mỗi ảnh')
    plt.xlabel('Số annotations')
    plt.ylabel('Số lượng ảnh')
    plt.grid(alpha=0.3)

    plt.tight_layout()
    plot_path = os.path.join(OUTPUT_DIR, 'dataset_statistics.png')
    plt.savefig(plot_path)
    plt.close()

    print(f"\nĐã lưu biểu đồ thống kê tại: {plot_path}")

    # Hiển thị biểu đồ
    display(plt.imread(plot_path))

    return {
        'num_images': num_images,
        'total_annotations': total_annotations,
        'avg_annotations_per_image': total_annotations / max(num_images, 1),
        'avg_text_length': sum(text_lengths) / len(text_lengths) if text_lengths else 0,
        'avg_bbox_size': sum(bbox_sizes) / len(bbox_sizes) if bbox_sizes else 0,
        'avg_confidence': sum(confidences) / len(confidences) if confidences else 0
    }


In [ ]:
def visualize_annotations(dataset=None, json_path=None, image_dir=None, output_dir=None, num_samples=5):
    """
    Hiển thị các annotations trên hình ảnh để kiểm tra trực quan

    Args:
        dataset: Dữ liệu JSON đã tải (ưu tiên)
        json_path: Đường dẫn đến file JSON chứa annotations (sử dụng nếu dataset=None)
        image_dir: Thư mục chứa hình ảnh
        output_dir: Thư mục để lưu hình ảnh với annotations
        num_samples: Số lượng mẫu muốn hiển thị (nếu None thì hiển thị tất cả)
    """
    # Đọc dataset nếu chưa cung cấp
    if dataset is None and json_path:
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                dataset = json.load(f)
        except Exception as e:
            print(f"Lỗi khi đọc file JSON: {str(e)}")
            return

    if not dataset:
        print("Không có dữ liệu để hiển thị!")
        return

    if not image_dir:
        print("Không có thư mục hình ảnh để hiển thị!")
        return

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # Lấy mẫu từ dataset nếu cần
    image_data_list = dataset["images"]
    if num_samples and num_samples < len(image_data_list):
        import random
        image_data_list = random.sample(image_data_list, num_samples)

    # Lặp qua từng hình ảnh
    for image_data in image_data_list:
        filename = image_data["file_name"]
        image_path = os.path.join(image_dir, filename)

        # Đọc hình ảnh
        try:
            img = cv2.imread(image_path)
            if img is None:
                print(f"Không thể đọc hình ảnh: {image_path}, bỏ qua.")
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"Lỗi khi đọc hình ảnh {filename}: {str(e)}")
            continue

        # Tạo figure với kích thước phù hợp
        plt.figure(figsize=(12, 12))
        plt.imshow(img)

        # Vẽ các annotations
        ax = plt.gca()
        for ann in image_data["annotations"]:
            bbox = ann["bbox"]
            text = ann["text"]

            # Vẽ bounding box
            x, y, w, h = bbox
            rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

            # Hiển thị text
            plt.text(x, y-5, text, color='red', fontsize=8,
                    backgroundcolor='white')

            # Nếu có polygon, vẽ đa giác chi tiết
            if "polygon" in ann:
                polygon = ann["polygon"]
                polygon_np = np.array(polygon)
                plt.plot(polygon_np[:, 0], polygon_np[:, 1], 'g-', linewidth=1)

        plt.title(f"Annotations for {filename}")
        plt.axis('off')

        # Lưu hình ảnh nếu cần
        if output_dir:
            output_path = os.path.join(output_dir, f"annotated_{filename}")
            plt.savefig(output_path, bbox_inches='tight')

        plt.show()
        plt.close()

# 5. Xuất các định dạng khác cho các framework OCR phổ biến
def convert_to_paddleocr_format(dataset, output_folder):
    """Chuyển đổi định dạng JSON sang định dạng PaddleOCR"""
    os.makedirs(output_folder, exist_ok=True)

    # Mỗi hình ảnh sẽ có một file txt riêng
    for image in dataset["images"]:
        filename = os.path.splitext(image["file_name"])[0] + ".txt"
        output_path = os.path.join(output_folder, filename)

        with open(output_path, 'w', encoding='utf-8') as f:
            for ann in image["annotations"]:
                # Sử dụng polygon nếu có, nếu không thì chuyển đổi từ bbox
                if "polygon" in ann:
                    points = ann["polygon"]
                else:
                    bbox = ann["bbox"]
                    x1, y1 = bbox[0], bbox[1]
                    x2, y2 = bbox[0] + bbox[2], bbox[1]
                    x3, y3 = bbox[0] + bbox[2], bbox[1] + bbox[3]
                    x4, y4 = bbox[0], bbox[1] + bbox[3]
                    points = [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]

                # Chuyển đổi thành chuỗi tọa độ
                points_str = ",".join([f"{int(p[0])},{int(p[1])}" for p in points])

                # Format: x1,y1,x2,y2,x3,y3,x4,y4,text
                line = f"{points_str},{ann['text']}\n"
                f.write(line)

    print(f"Đã chuyển đổi sang định dạng PaddleOCR tại {output_folder}")


In [ ]:
def edit_annotations(dataset, image_dir, output_json=None):
    """
    Hiển thị và cho phép chỉnh sửa các annotations đơn giản

    Args:
        dataset: Dataset gốc cần chỉnh sửa
        image_dir: Thư mục chứa hình ảnh
        output_json: Đường dẫn để lưu dataset sau khi chỉnh sửa
    """
    from IPython.display import clear_output
    import pandas as pd
    from ipywidgets import widgets

    # Lấy danh sách tất cả annotations
    all_annotations = []
    for img_idx, image_data in enumerate(dataset["images"]):
        filename = image_data["file_name"]
        for ann_idx, ann in enumerate(image_data["annotations"]):
            all_annotations.append({
                'img_idx': img_idx,
                'ann_idx': ann_idx,
                'filename': filename,
                'text': ann['text'],
                'confidence': ann.get('confidence', 0)
            })

    # Tạo DataFrame để hiển thị và chỉnh sửa
    df = pd.DataFrame(all_annotations)

    # Sắp xếp theo confidence tăng dần (những text có confidence thấp có thể cần chỉnh sửa)
    df = df.sort_values('confidence')

    # Hiển thị DataFrame
    print("Danh sách annotations (sắp xếp theo confidence tăng dần):")
    display(df)

    # Tạo widget để chỉnh sửa
    print("\nChỉnh sửa annotation (nhập img_idx, ann_idx và text mới):")

    img_idx_input = widgets.IntText(description='img_idx:')
    ann_idx_input = widgets.IntText(description='ann_idx:')
    text_input = widgets.Text(description='Text mới:', style={'description_width': 'initial'})
    submit_button = widgets.Button(description='Cập nhật')

    display(img_idx_input, ann_idx_input, text_input, submit_button)

    output = widgets.Output()
    display(output)

    def on_submit_button_clicked(b):
        with output:
            clear_output()
            img_idx = img_idx_input.value
            ann_idx = ann_idx_input.value
            new_text = text_input.value

            try:
                # Cập nhật text trong dataset
                dataset["images"][img_idx]["annotations"][ann_idx]["text"] = new_text
                print(f"Đã cập nhật: img_idx={img_idx}, ann_idx={ann_idx}, text='{new_text}'")

                # Lưu dataset nếu cần
                if output_json:
                    with open(output_json, 'w', encoding='utf-8') as f:
                        json.dump(dataset, f, ensure_ascii=False, indent=2)
                    print(f"Đã lưu dataset tại {output_json}")
            except IndexError:
                print("Lỗi: img_idx hoặc ann_idx không hợp lệ!")
            except Exception as e:
                print(f"Lỗi: {str(e)}")

    submit_button.on_click(on_submit_button_clicked)

    # Hiển thị ảnh để kiểm tra
    print("\nKiểm tra hình ảnh (nhập img_idx để hiển thị):")

    img_view_input = widgets.IntText(description='img_idx:')
    view_button = widgets.Button(description='Hiển thị')

    display(img_view_input, view_button)

    view_output = widgets.Output()
    display(view_output)

    def on_view_button_clicked(b):
        with view_output:
            clear_output()
            img_idx = img_view_input.value

            try:
                image_data = dataset["images"][img_idx]
                filename = image_data["file_name"]
                image_path = os.path.join(image_dir, filename)

                # Đọc hình ảnh
                img = cv2.imread(image_path)
                if img is None:
                    print(f"Không thể đọc hình ảnh: {image_path}")
                    return
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                # Hiển thị hình ảnh với annotations
                plt.figure(figsize=(12, 12))
                plt.imshow(img)

                # Vẽ các annotations
                ax = plt.gca()
                for ann_idx, ann in enumerate(image_data["annotations"]):
                    bbox = ann["bbox"]
                    text = ann["text"]

                    # Vẽ bounding box
                    x, y, w, h = bbox
                    rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none')
                    ax.add_patch(rect)

                    # Hiển thị text và ann_idx
                    plt.text(x, y-5, f"{ann_idx}: {text}", color='red', fontsize=10,
                           backgroundcolor='white')

                plt.title(f"Annotations for {filename} (img_idx={img_idx})")
                plt.axis('off')
                plt.show()
            except IndexError:
                print("Lỗi: img_idx không hợp lệ!")
            except Exception as e:
                print(f"Lỗi: {str(e)}")

    view_button.on_click(on_view_button_clicked)

In [ ]:
print("===== OCR AUTO LABELING VÀ ĐÁNH GIÁ =====")
print("1. Tự động gán nhãn với PaddleOCR (Tiếng Việt)")
print("2. Tự động gán nhãn với PaddleOCR (Tiếng Anh)")
print("3. Hiển thị và đánh giá bộ dataset")
print("4. Chỉnh sửa annotations")
print("5. Xuất định dạng PaddleOCR")

# Định nghĩa đường dẫn đầu ra
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_json = os.path.join(OUTPUT_DIR, f'ocr_dataset_{timestamp}.json')
visualize_output = VISUALIZE_DIR

# Chạy gán nhãn tự động với ngôn ngữ Tiếng Việt mặc định
print("\nBắt đầu gán nhãn tự động với PaddleOCR (Tiếng Việt)...")
dataset = auto_label_with_paddleocr(IMAGE_DIR, output_json, lang='vi', visualize_dir=visualize_output)

# Đánh giá dataset
print("\nĐánh giá và thống kê dataset...")
stats = evaluate_dataset(dataset)

# Hiển thị một số mẫu annotation
print("\nHiển thị một số mẫu annotations...")
visualize_annotations(dataset, image_dir=IMAGE_DIR, output_dir=visualize_output, num_samples=3)

# Thông báo hoàn tất
print("\n===== HOÀN TẤT =====")
print(f"Dataset đã được lưu tại: {output_json}")
print(f"Visualizations đã được lưu tại: {visualize_output}")
print("Bạn có thể chỉnh sửa annotations bằng cách chạy hàm edit_annotations(dataset, IMAGE_DIR, output_json)")